In [ ]:
!python -m pip install -Uq pandas==1.3.0

In [ ]:
import json
import requests
import os

import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as mtick
%matplotlib inline

import seaborn as sns

import sagemaker
import boto3

In [ ]:
pd.__version__

In [ ]:
pd.set_option('display.max_columns', None)
sns.set_palette("Spectral")
size=15
params = {'legend.fontsize': 'large',
          'figure.figsize': (20,8),
          'axes.labelsize': size,
          'axes.titlesize': size,
          'xtick.labelsize': size*0.75,
          'ytick.labelsize': size*0.75,
          'axes.titlepad': 25}
plt.rcParams.update(params);

# Connect to SageMaker Workspace

In [ ]:
s3client = boto3.client('s3')
sess = sagemaker.Session()
bucket = sess.default_bucket() 
folder_prefix = 'opensource-10k-data'
print(f"Default bucket: {bucket}")

In [ ]:
# import credentials saved in a local only file 
with open('./config.json') as f:
    creds = json.load(f)


# Clean and Transform Data

### Create company profiles df

In [ ]:
dfs = [] # an empty list to store the data frames

data_dir = 'data/company_profiles'
for f in [os.path.join(data_dir, p) for p in os.listdir(data_dir)]:
    data = pd.read_json(f) # read data frame from json file
    dfs.append(data) # append the data frame to the list

company_profiles = pd.concat(dfs, ignore_index=True)

### Create income sheets df

In [ ]:
dfs = [] # an empty list to store the data frames

data_dir = 'data/income_sheets'
for file in [os.path.abspath(os.path.join(data_dir, p)) for p in os.listdir(data_dir) if not p.startswith('.')]:
    data = pd.read_json(file) # read data frame from json file
    dfs.append(data) # append the data frame to the list

income_sheets = pd.concat(dfs, ignore_index=True)
income_sheets['fillingDate'] = pd.to_datetime(income_sheets['fillingDate'])
income_sheets['year'] = (income_sheets['fillingDate'] - pd.DateOffset(years=1)).dt.year

In [ ]:
income_sheets.head()

### Create balance sheets df

In [ ]:
dfs = [] # an empty list to store the data frames

data_dir = 'data/balance_sheets'
for file in [os.path.abspath(os.path.join(data_dir, p)) for p in os.listdir(data_dir) if not p.startswith('.')]:
    data = pd.read_json(file) # read data frame from json file
    dfs.append(data) # append the data frame to the list

balance_sheets = pd.concat(dfs, ignore_index=True)
balance_sheets['fillingDate'] = pd.to_datetime(balance_sheets['fillingDate'])
balance_sheets['year'] = (balance_sheets['fillingDate'] - pd.DateOffset(years=1)).dt.year

In [ ]:
balance_sheets.head()

### Create cash flow df

In [ ]:
dfs = [] # an empty list to store the data frames

data_dir = 'data/cash_flow_statements'
for file in [os.path.abspath(os.path.join(data_dir, p)) for p in os.listdir(data_dir) if not p.startswith('.')]:
    data = pd.read_json(file) # read data frame from json file
    dfs.append(data) # append the data frame to the list

cashflow_dataset = pd.concat(dfs, ignore_index=True)
cashflow_dataset['fillingDate'] = pd.to_datetime(cashflow_dataset['fillingDate'])
cashflow_dataset['year'] = (cashflow_dataset['fillingDate'] - pd.DateOffset(years=1)).dt.year

In [ ]:
cashflow_dataset.head()

# Create main df

In [ ]:
df = income_sheets[['year','symbol','netIncome','revenue','incomeTaxExpense']]\
        .merge(balance_sheets[['year','symbol','totalStockholdersEquity']], on=['year','symbol'], how='left')\
        .merge(cashflow_dataset[['year','symbol','dividendsPaid','operatingCashFlow',
                                 'depreciationAndAmortization','accountsReceivables','accountsPayables',
                                 'netCashUsedForInvestingActivites','freeCashFlow'
                                ]], on=['year','symbol'], how='left')
df = df.rename(columns={
    'netIncome':'net_income',
    'revenue':'sales',
    'totalStockholdersEquity':'book_value',
    'incomeTaxExpense':'income_tax',
    'accountsReceivables':'net_accounts_receivable',
    'accountsPayables':'net_accounts_payable',
    'netCashUsedForInvestingActivites':'maintenance_capital_expenditure'
})
df['book_value_plus_dividends'] = df['book_value'] + df['dividendsPaid']
df['year_datetime'] = pd.to_datetime(df['year'].astype(str) + '/12/31', yearfirst=True)
df = df.sort_values(by=['symbol','year'])

In [ ]:
df

# Calculate the Big Four Growth Rates
pg. 131 Invested

1. Net Income (Net Profit or Net Earnings)
2. Book Value (Equity or Shareholder Equity) + Dividends
3. Sales (Revenue)
4. Operating Cash

Then calculate the growth rate (aka percent change) of each

In [ ]:
# Calculate 4 Growth Rates
df['net_income_growth'] = df.groupby(['symbol'])['net_income'].pct_change()
df['book_value_dividends_growth'] = df.groupby(['symbol'])['book_value_plus_dividends'].pct_change()
df['sales_growth'] = df.groupby(['symbol'])['sales'].pct_change()
df['cash_flow_growth'] = df.groupby(['symbol'])['operatingCashFlow'].pct_change()

In [ ]:
# visualize growth rates over time
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(nrows=2, ncols=2, sharey=False, figsize=(15,8))

sns.lineplot(data=df, x='year_datetime', y='net_income_growth', hue='symbol', ax=ax1)
sns.lineplot(data=df, x='year_datetime', y='book_value_dividends_growth', hue='symbol', ax=ax2)
sns.lineplot(data=df, x='year_datetime', y='sales_growth', hue='symbol', ax=ax3)
sns.lineplot(data=df, x='year_datetime', y='cash_flow_growth', hue='symbol', ax=ax4)

# Define the date format
date_form = mdates.DateFormatter("%Y")
for ax in [ax1, ax2, ax3, ax4]:
    ax.yaxis.set_major_formatter(mtick.PercentFormatter(xmax=1.0))
    ax.xaxis.set_major_locator(mdates.YearLocator(base=1, month=12, day=31))
    ax.xaxis.set_major_formatter(date_form)
    ax.set_xlabel('year', fontsize=10)


# Calculate the Windage Growth Rate
or the one growth rate to use for subsequent analysis

pg. 133 Invested

In [ ]:
# Compare across growth rates
growth_rates = \
df.groupby('symbol').agg({'net_income_growth':['mean','std','median'],
                          'book_value_dividends_growth':['mean','std','median'],
                          'sales_growth':['mean','std','median'],
                          'cash_flow_growth':['mean','std','median']
                         })

# Calculate the overall growth rate we'll use in analysis as an average between our mean and medians for past several years
# and use only 90% of that windage estimate and round down to the nearest ones/single digit percentage (being conservative in our estimate for growth)
growth_rates['windage_growth_rate'] = \
growth_rates[[('net_income_growth', 'mean'),('net_income_growth', 'median'),
              ('book_value_dividends_growth', 'mean'),('book_value_dividends_growth', 'median'),
              ('sales_growth', 'mean'),('sales_growth', 'median'),
              ('cash_flow_growth','mean'), ('cash_flow_growth', 'median')
             ]].mean(axis=1).round(decimals=2)*0.90

In [ ]:
growth_rates.sort_values(by=['windage_growth_rate'], ascending=False)

*^ Do these windage growth rates look reasonable? How do they compare with analysts' forecasts?*
- 

# Calculate Valuations

## Owners Earnings for Ten Cap
pg. 194 Invested  

Net Income   
\+ Depreciation & Amortization   
\+ Net Change in Accounts Receivables   
\+ Net Change in Accounts Payable   
\+ Income Tax   
\+ Maintainance Capital Expenditures  
= Owner Earnings

In [ ]:
def owner_earnings_cap(row, cap=10):
    return (row['net_income'] \
         + row['depreciationAndAmortization'] \
         + row['net_accounts_receivable'] \
         + row['net_accounts_payable'] \
         + row['income_tax'] \
         + row['maintenance_capital_expenditure']) * cap

In [ ]:
df['owner_earnings_ten_cap'] = df.apply(lambda row: owner_earnings_cap(row, 10), axis=1)

In [ ]:
df.loc[df.reset_index().groupby(['symbol'])['year'].idxmin()][['year','symbol','owner_earnings_ten_cap']].sort_values(by='owner_earnings_ten_cap', ascending=False)

## Payback Time
pg. 201 Invested

Free Cash Flow grown by the compunded Windage Growth Rate for 8 years

In [ ]:
free_cash_flow = df.loc[df.reset_index().groupby(['symbol'])['year'].idxmin()][['year','symbol','freeCashFlow']]
free_cash_flow = free_cash_flow.merge(growth_rates[['windage_growth_rate']].reset_index(), on='symbol').rename(columns={('windage_growth_rate',''): 'windage_growth_rate'}).copy()

In [ ]:
def freeCashFlow_analysis(year_0_free_cash_flow, windage_growth_rate):
    free_cash_flow = [year_0_free_cash_flow]
    expected_growth_in_freeCashFlow = []
    cumu_free_cash_flow = [0]
    
    # payback time in 8 years (+1 for year 0)
    for year in range(9):
        new_growth_in_cash_flow = free_cash_flow[-1] * windage_growth_rate

        expected_growth_in_freeCashFlow.append(new_growth_in_cash_flow)
        free_cash_flow.append(free_cash_flow[-1] + new_growth_in_cash_flow)
        cumu_free_cash_flow.append(cumu_free_cash_flow[-1] + free_cash_flow[-1])
        
    return free_cash_flow[:-1], expected_growth_in_freeCashFlow, cumu_free_cash_flow[:-1]

In [ ]:
free_cash_flow_analysis_dfs = []

for symbol in free_cash_flow['symbol'].values:
    symbol_df = free_cash_flow.loc[free_cash_flow['symbol']==symbol]
    
    free_cash_flow_lst, \
    expected_growth_in_freeCashFlow_lst, \
    cumu_free_cash_flow_lst = freeCashFlow_analysis(symbol_df['freeCashFlow'].item(), symbol_df['windage_growth_rate'].item())
    year = symbol_df['year'].item()
                 
    symbol_df = pd.DataFrame({'free_cash_flow':free_cash_flow_lst,
                      'expected_growth_in_freeCashFlow':expected_growth_in_freeCashFlow_lst,
                      'cumu_free_cash_flow':cumu_free_cash_flow_lst,
                      'year': range(year, year+9)
                      })
    symbol_df['symbol'] = symbol
    
    free_cash_flow_analysis_dfs.append(symbol_df)
    

free_cash_flow_analysis = pd.concat(free_cash_flow_analysis_dfs)

In [ ]:
idx = free_cash_flow_analysis.groupby(['symbol'])['year'].transform(max) == free_cash_flow_analysis['year']
free_cash_flow_analysis[idx][['symbol','year','cumu_free_cash_flow']].sort_values(by='cumu_free_cash_flow', ascending=False)